# Connect MoveIt with Real Actuators

Now we have to combine what we learned in **Episode 1 and Episode 2**, to through moveit, make the robot move.
Just a fast reminder of the main commands of previous episodes:

## Create the controllers for talking with the real robot

In Episode 1 we talked about the position control, and how with a **python script** that we created called **move_openmanipulator.py** in package **openmanipulator_morpheus_chair_tutorials**, we could move the robot to any position.

### Create The TrajectoryServer

The most standard way to use moveit is through TrajectoryJointControllers. Its essentially the cotrollers that can be setup automatically "in theory" in the **Moveit! Wizard**.

So Movit! will send goals to an action server of message type **control_msgs/FollowJointTrajectoryAction**. And its this **trajectory_server** that is going to recieve the goal and give the feedback to moveit. 

In [ ]:
roscd open_manipulator_core/launch
ls controller_launch.launch

<p style="background:#3B8F10;color:white;" id="simple_python">**Launch File: controller_launch.launch** </p>

In [ ]:
<launch>
   <arg name="device_name"                default="/dev/ttyUSB0"/>
   <arg name="baud_rate"                  default="3000000"/>
   <arg name="latency_timer"              default="1"/>
   <arg name="scan_range"                 default="10"/>

   <arg name="profile_velocity"           default="75"/>
   <arg name="profile_acceleration"       default="75"/>

   <param name="device_name"              value="$(arg device_name)"/>
   <param name="baud_rate"                value="$(arg baud_rate)"/>
   <param name="latency_timer"            value="$(arg latency_timer)"/>

   <param name="scan_range"               value="$(arg scan_range)"/>

   <param name="profile_velocity"         value="$(arg profile_velocity)"/>
   <param name="profile_acceleration"     value="$(arg profile_acceleration)"/>
   
   <node name="position_control" pkg="dynamixel_workbench_controllers" type="position_control" required="true" output="screen"/>
   
   
   <node name="dynamixel_action_server" pkg="open_manipulator_core" type="trajectory_server" required="true" output="screen" />
   <node name="gripper_commander" pkg="open_manipulator_core" type="gripper_commander" required="true" output="screen" /> 
 </launch>


<p style="background:#3B8F10;color:white;" id="simple_python">**END Launch File: controller_launch.launch** </p>

As you see is very similar to the **position_control.launch** in **dynamixel_workbench_controllers** that we used in Episode 1. In fact its launching possition control. **BUT**, we are launching **TWO** launch files:
* trajectory_server, from package open_manipulator_core: This is the one that is responsible for creating the ActionServer of type **control_msgs/FollowJointTrajectory**. This will convert **FollowJointTrajectory** messages into **position of the joints of the robot** ( what we did in episode 1).
* gripper_commander, from package open_manipulator_core: This is a topic subscriber of type std_msgs/String that wil be in charge of converting Strings of **open_gripper** and **close_gripper** into its corresponding gripper position values.

So lets have a look how this **ActionServer** is made with Cpp:

<p style="background:#3B8F10;color:white;" id="simple_python">**Cpp: trajectory_server.cpp** </p>

[trajectory_server.cpp](extra_files/trajectory_server.cpp)

<p style="background:#3B8F10;color:white;" id="simple_python">**END Cpp: trajectory_server.cpp** </p>

We wont go into detail but only note the following basics:

Here we Start a publishier into our known topic for moving the angles of the robot through position:

In [ ]:
jointPub =nh_.advertise<sensor_msgs::JointState>("goal_dynamixel_position", 1);

And here we start an action server of message type **FollowJointTrajectory**

In [ ]:
actionlib::SimpleActionServer<control_msgs::FollowJointTrajectoryAction> as_;
as_(nh_, name,boost::bind(&DynamixelTrajectoryAction::executeCB, this, _1), false)

jointSub = nh_.subscribe<sensor_msgs::JointState>(
      "joint_states", 1,
      boost::bind(&DynamixelTrajectoryAction::onJointState, this, _1));
as_.start();

In [ ]:
DynamixelTrajectoryAction::executeCB
{
... Among loads of things related to giving feedback to the action and other details
...
    jointPub.publish(targetState); # It publishes the state that we want to move the joints to.
}


Each robot will do it slightly different but the structure to be used for **MoveIt!** has to be exactly the same, in order to make it work.

We will talk about the gripper when we need it, for the moment its enough.

## Launch the new Control

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in OpenManipulatorConnecter PC #1</p>
</th>
</tr>
</table>
<br>

In [ ]:
roslaunch open_manipulator_core controller_launch.launch

When launching it you should see, among other things these topics:
* /arm/follow_joint_trajectory/cancel
* /arm/follow_joint_trajectory/feedback
* /arm/follow_joint_trajectory/goal
* /arm/follow_joint_trajectory/result
* /arm/follow_joint_trajectory/status

* /gripper_command

Stay with thes enames because we will need them for the next step which is configure **Moveit** to move the real robot:

## Configure MoveIt! to be able to use the **FollowJointTrajectory** action server

Lets start form the **movit package** we created in **Episode 2**, called **openmanipulator_ep2_movit_config**. You can find it in the repo <a href="https://bitbucket.org/theconstructcore/openmanipulator_morpheus_chair/src/master/">openmanipulator_morpheus_chair</a>.

You will have to edit and create the following files like so:

In [ ]:
roscd openmanipulator_ep2_movit_config
touch config/controllers.yaml
touch config/joint_names.yaml
touch launch/open_manipulator_planning_execution.launch

<p style="background:#3B8F10;color:white;" id="simple_python">**controllers.yaml** </p>

In [ ]:
controller_list:
  - name: "arm"
    action_ns: follow_joint_trajectory
    type: FollowJointTrajectory
    joints: [id_1, id_2, id_3, id_4, id_5, id_6]

If you remember we already created a **ros_controllers.yaml** through the **MoveIt1 Wizard**. But its more combersome to use and cluttered with stuff that we dont need.
So we create a clean and simple one that contains only the joints, the type, name and action_ns.

Here is where the previous section comes in handy: **/arm/follow_joint_trajectory/goal**:
* name:arm
* action_ns: follow_joint_trajectory
* type: Is the Base type of the FollowJointTrajectoryAction
* joints: the same as the ones stated in the **ros_controllers.yaml**.

<p style="background:#3B8F10;color:white;" id="simple_python">**joint_names.yaml** </p>

In [ ]:
controller_joint_names: [id_1, id_2, id_3, id_4, id_5, id_6, grip_joint, grip_joint_sub]

<p style="background:#3B8F10;color:white;" id="simple_python">**open_manipulator_planning_execution.launch** </p>

In [ ]:
<launch>

  <rosparam command="load" file="$(find openmanipulator_ep2_movit_config)/config/joint_names.yaml"/>

  <include file="$(find openmanipulator_ep2_movit_config)/launch/planning_context.launch" >
    <arg name="load_robot_description" value="true" />
  </include>

  <node name="joint_state_publisher" pkg="joint_state_publisher" type="joint_state_publisher">
    <param name="/use_gui" value="false"/>
    <rosparam param="/source_list">[/joint_states]</rosparam>
  </node>

  <include file="$(find openmanipulator_ep2_movit_config)/launch/move_group.launch">
    <arg name="publish_monitored_planning_scene" value="true" />
  </include>

  <include file="$(find openmanipulator_ep2_movit_config)/launch/moveit_rviz.launch">
    <arg name="config" value="true"/>
  </include>

</launch>

We also have to edit the following files:
* launch/open_manipulator_moveit_controller_manager.launch.xml: This file already was created but it was empty.

<p style="background:#3B8F10;color:white;" id="simple_python">**open_manipulator_moveit_controller_manager.launch.xml** </p>

In [ ]:
<launch>
  <rosparam file="$(find openmanipulator_ep2_movit_config)/config/controllers.yaml"/>
  <param name="use_controller_manager" value="false"/>
  <param name="trajectory_execution/execution_duration_monitoring" value="false"/>
  <param name="moveit_controller_manager" value="moveit_simple_controller_manager/MoveItSimpleControllerManager"/>
</launch>


Here we **load the controllers.yaml**. And this launch is called through the following pipeline:
* 0) openmanipulator_ep2_movit_config/launch/**open_manipulator_planning_execution.launch** **Calls** openmanipulator_ep2_movit_config/launch/**move_group.launch**
* 1) openmanipulator_ep2_movit_config/launch/**move_group.launch** **Calls** openmanipulator_ep2_movit_config/launch/**trajectory_execution.launch.xml**
* 2) openmanipulator_ep2_movit_config/launch/**trajectory_execution.launch.xml** **Calls** openmanipulator_ep2_movit_config/launch/**open_manipulator_moveit_controller_manager.launch.xml**

## Launch MoveIt! With the controllers to be used defined

Now we have to connect ROSDS to the Manipulator. For that we use like always the **RealRobot Connection**. 

**IF YOU HAD CLOSED IT, relaunch again the controllers in the real robot:**

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in OpenManipulatorConnecter PC #1</p>
</th>
</tr>
</table>
<br>

In [ ]:
roslaunch open_manipulator_core controller_launch.launch

And Start Moveit!

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in ROSDS WebShell #1</p>
</th>
</tr>
</table>
<br>

In [ ]:
roslaunch openmanipulator_ep2_movit_config open_manipulator_planning_execution.launch

You should now be able to see in the **Graphical Interface** or in your local computer if you executed everything there, the RVIZ with the **OpneManipulator** in the position where it is in reality:

<img src="images/IMG_6381.jpg"/>

<img src="images/rviz_controllers.png"/>

Now if you execute a plan and execute, the real robot should move:

You can see how the goals and feedback are sent:

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in ROSDS WebShell or OpenManipulatorPC  #2</p>
</th>
</tr>
</table>
<br>

In [ ]:
reset;rostopic echo /arm/follow_joint_trajectory/goal

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in ROSDS WebShell or OpenManipulatorPC  #3</p>
</th>
</tr>
</table>
<br>

In [ ]:
reset;rostopic echo /arm/follow_joint_trajectory/goal